In [1]:
import pandas as pd
import numpy as np
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
from sklearn.cluster import KMeans 
import matplotlib.cm as cm
import matplotlib.colors as colors

## Read html to csv

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url)[0]
df.to_csv()
df.head()

## Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned

In [ ]:
cols=[x for i,x in enumerate(df.index) if df.iat[i,1]=='Not assigned']
df1= df.drop(cols,axis=0)
#df1=df[~df['Borough'].isin(['Not assigned'])] ~代表取反
#df1.head()
df1.reset_index(drop=True,inplace=True)

## If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [ ]:
cols1=[ i for i,x in enumerate(df1.index) if df1.iat[i,2] == 'Not assigned']
for i in cols1:
    df1.replace(df1.iloc[i,2],df1.iloc[i,1],inplace=True)


## In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [ ]:
df1.shape

## Build the dataframe with coordinate

In [ ]:
df_cor = pd.read_csv('http://cocl.us/Geospatial_data')
df_cor.head()

In [ ]:
df2=pd.merge(df1,df_cor,how='inner',on=['Postal Code'])
df2.head()

## Create map of Toronto using latitude and longitude values

In [ ]:
map_can = folium.Map(location=[43.7, -79.25], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_can)
    
map_can

## Cluster those location by K-means using the Latitude and Londtitude

In [ ]:

X= df2.values[:,3:]


In [ ]:

k_means = KMeans(init = "k-means++", n_clusters = 3, n_init = 10)
k_means.fit(X)
k_means_labels = k_means.labels_
k_means_labels


In [ ]:
# add clustering labels
df2.insert(0, 'Cluster Labels', k_means_labels)
df2


In [ ]:
# create map
map_clusters = folium.Map(location=[43.7, -79.25], zoom_start=11)

# set color scheme for the clusters
x = np.arange(3)
ys = [i + x + (i*x)**2 for i in range(3)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df2['Latitude'], df2['Longitude'], df2['Neighbourhood'], df2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters